In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install open_clip_torch matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
import open_clip
import torch
import torch.nn.functional as F
from PIL import Image
import json, os
from pathlib import Path
from tqdm import tqdm
import numpy as np

In [ ]:
# --- Device / helpers ---
device = "cuda" if torch.cuda.is_available() else "cpu"
amp_enabled = (device == "cuda")

In [ ]:
# --- Setup (paths) ---
ANN_PATH = "/content/drive/MyDrive/data/nocap_val_4500_captions.json"
IMG_DIR = "/content/drive/MyDrive/data/images"
CAPTION_JSON = "/content/drive/MyDrive/data/captions.json"
CAPTION_CLIPSCORE_JSON = "/content/drive/MyDrive/data/captions_with_clipscores.json"
CAPTION_HYBRID_JSON = "/content/drive/MyDrive/data/captions_hybrid_scored.json"

In [ ]:
def clean_caption(tokens):
    # open_clip.decode -> string with special tokens; strip them
    s = open_clip.decode(tokens).split("<end_of_text>")[0]
    s = s.replace("<start_of_text>", "").strip()
    # normalize whitespace
    return " ".join(s.split())

# --- Model variants you requested ---
VARIANTS = [
    ("coca_ViT-B-32", "laion2b_s13b_b90k"),
    ("coca_ViT-B-32", "mscoco_finetuned_laion2b_s13b_b90k"),
    ("coca_ViT-L-14", "laion2b_s13b_b90k"),
    ("coca_ViT-L-14", "mscoco_finetuned_laion2b_s13b_b90k"),
]

# --- Load all models & transforms once (to avoid reloading per image) ---
models = []
for model_name, ckpt in VARIANTS:
    model, _, transform = open_clip.create_model_and_transforms(
        model_name=model_name,
        pretrained=ckpt
    )
    model = model.to(device)
    model.eval()
    models.append((model_name, ckpt, model, transform))

print(f"Loaded {len(models)} CoCa variants on {device}.")

open_clip_pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/2.55G [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/2.55G [00:00<?, ?B/s]

Loaded 4 CoCa variants on cpu.


In [ ]:
# --- Read nocaps json ---
LIMIT = 1000

with open(ANN_PATH, "r") as f:
    nocaps = json.load(f)

images = nocaps.get("images", [])
if LIMIT is not None:
    images = images[:LIMIT]

print(f"Will caption {len(images)} image(s).")

Will caption 1000 image(s).


In [ ]:
# --- Generate captions ---
results = []
missing_files = 0
failed = 0

for img_info in tqdm(images, desc="Captioning"):
    fname = img_info["file_name"]
    fpath = Path(IMG_DIR) / fname
    if not fpath.exists():
        missing_files += 1
        continue

    captions = []
    try:
        with Image.open(fpath).convert("RGB") as pil:
            for (model_name, ckpt, model, transform) in models:
                # transform and move to device
                pixel = transform(pil).unsqueeze(0).to(device)
                with torch.no_grad(), torch.cuda.amp.autocast(enabled=amp_enabled):
                    tokens = model.generate(pixel)
                cap = clean_caption(tokens[0])
                captions.append(cap)
                print(cap)
    except Exception as e:
        failed += 1
        # keep going; skip this image
        continue

    results.append({
        "file_name": fname,
        "captions": captions
    })

Captioning:   0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipython-input-186465145.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=amp_enabled):


this photo is one of the first photos i have of my great - great - great great great great great great great great great great great great great
a little boy that is standing up with a bat .
1 9 5 0 - 0 4 - 0 1 - baby - in - front - of - house - 0 1 . jpg


Captioning:   0%|          | 1/1000 [02:55<48:43:57, 175.61s/it]

an old black and white photo of a little boy .
an elephant in the national park in south india stock photo - 2 2 2 2 2 2 2 2
an elephant with a seat on its back .
elephant in chiang mai , northern thailand


Captioning:   0%|          | 2/1000 [04:48<38:23:39, 138.50s/it]

...

a camera on top of a tripod with a camera attached to it .
a white flower with a white flower in front of a white background stock photo - 1 3 2 2 2 2 2 2
a pair of shoes that have a cat in them .
white easter lily flower on a pile of silver coins stock fotó - 1 1 9 7 4 3 1 8 0


Captioning:   2%|▏         | 20/1000 [43:47<38:12:29, 140.36s/it]

a white flower sitting on top of coins .
2 0 1 1 1 1 2 2 - 1 1 2 2 1 1 - u 1 8 u 1 8 u 1 8 u 1 8 u
a soccer player is trying to control the ball .
soccer player kicking the soccer ball on the field


Captioning:   2%|▏         | 20/1000 [45:30<37:09:36, 136.51s/it]


KeyboardInterrupt: 

In [ ]:
# --- Save ---
with open(CAPTION_JSON, "w") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\nDone. Saved {len(results)} items to: {CAPTION_JSON}")
print(f"Missing files: {missing_files} | Failed during generation: {failed}")
